In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load models
harassment_model_path = "harass.h5"
harassment_model = tf.keras.models.load_model(harassment_model_path)

gender_model_path = "gender_classification_model.h5"
gender_model = tf.keras.models.load_model(gender_model_path)

expression_model_path = "emotion_model.h5"  # Load the emotion model
expression_model = tf.keras.models.load_model(expression_model_path)

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to process each frame for harassment detection
def process_frame_for_harassment(frame):
    temp_img_path = "temp_frame.jpg"
    cv2.imwrite(temp_img_path, frame)
    img = load_img(temp_img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    features = features.reshape(1, 7 * 7 * 512)
    predictions = harassment_model.predict(features)
    return np.argmax(predictions[0])

def process_frame_for_gender(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    male_count = 0
    female_count = 0
    lone_woman_detected = False
    detected_emotions = []  # Initialize an empty list to store detected emotions

    for (x, y, w, h) in faces:
        margin = int(max(w, h) * 0.3)
        x1, y1 = max(0, x - margin), max(0, y - margin)
        x2, y2 = min(frame.shape[1], x + w + margin), min(frame.shape[0], y + h + margin)
        
        roi = frame[y1:y2, x1:x2]
        roi_resized = cv2.resize(roi, (64, 64))
        roi_array = img_to_array(roi_resized) / 255.0
        roi_array = np.expand_dims(roi_array, axis=0)
        
        prediction = gender_model.predict(roi_array)
        result = "Male" if prediction[0][0] >= 0.5 else "Female"
        
        if result == "Male":
            male_count += 1
        else:
            female_count += 1
            if male_count == 0:
                lone_woman_detected = True
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Gender label above the bounding box
        label = f"{result} ({prediction[0][0]:.2f})"
        label_position = (x1, y1 - 10) if y1 - 10 > 10 else (x1, y2 + 30)
        cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, male_count, female_count, lone_woman_detected, detected_emotions

def process_frame_for_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    detected_emotions = []  # Initialize an empty list to store detected emotions

    for (x, y, w, h) in faces:
        margin = int(max(w, h) * 0.3)
        x1, y1 = max(0, x - margin), max(0, y - margin)
        x2, y2 = min(frame.shape[1], x + w + margin), min(frame.shape[0], y + h + margin)
        
        face_roi = gray[y1:y2, x1:x2]
        face_roi_resized = cv2.resize(face_roi, (48, 48))
        face_roi_normalized = face_roi_resized / 255.0
        face_roi_normalized = face_roi_normalized.astype(np.float32)
        face_roi_rgb = cv2.cvtColor(face_roi_normalized, cv2.COLOR_GRAY2RGB)
        face_roi_expanded = np.expand_dims(face_roi_rgb, axis=0)
        
        try:
            emotion_prediction = expression_model.predict(face_roi_expanded)
            emotion_label = emotion_labels[np.argmax(emotion_prediction)]
            detected_emotions.append(emotion_label)
            
            # Emotion label below the bounding box
            label_position = (x1, y2 + 30) if y2 + 30 < frame.shape[0] else (x1, y1 - 10)
            cv2.putText(frame, emotion_label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except Exception as e:
            print("Error during emotion prediction:", str(e))
    
    return frame, detected_emotions

def process_video_from_camera():
    cap = cv2.VideoCapture(0)  # Use camera index 0 for the default camera
    frame_interval = 1  # Adjust frame interval as needed

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        harassment_prediction = process_frame_for_harassment(frame)
        frame, male_count, female_count, lone_woman_detected, detected_emotions = process_frame_for_gender(frame)
        frame, detected_emotions = process_frame_for_emotion(frame)
        
        # Check for harassment based on detected emotions
        emotion_criteria = ['Disgust', 'Angry', 'Fear']
        harassment_detected = any(emotion in detected_emotions for emotion in emotion_criteria)
        
        # Updated Criteria for Harassment Detection
        if male_count > 0 and female_count > 0 and harassment_prediction == 1:
            if harassment_detected:
                cv2.putText(frame, "!!! HARASSMENT DETECTED !!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
        
        if lone_woman_detected and male_count == 0:
            cv2.putText(frame, "!!! LONE WOMAN DETECTED !!!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4)
        
        if lone_woman_detected and male_count > 0:
            cv2.putText(frame, f"LONE WOMAN SURROUNDED BY {male_count} MEN", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)

        cv2.imshow('Harassment, Gender, and Emotion Detection', frame)

        if cv2.waitKey(frame_interval) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Run the function to process the camera feed
process_video_from_camera()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━